# Using Target Pixel Files with Lightkurve

## Learning Goals

By the end of this tutorial, you will:

- Be able to download and plot target pixel files from the data archive using [Lightkurve](https://docs.lightkurve.org).
- Be able to access target pixel file metadata.
- Understand where to find more details about *Kepler* target pixel files.


## Introduction

The [*Kepler*](https://www.nasa.gov/mission_pages/kepler/main/index.html), [*K2*](https://www.nasa.gov/mission_pages/kepler/main/index.html), and [*TESS*](https://tess.mit.edu/) telescopes observe stars for long periods of time, from just under a month to four years. By doing so they observe how the brightnesses of stars change over time.

*Kepler* selected certain pixels around targeted stars to be downloaded from the spacecraft. These were stored as *target pixel files* that contain data for each observed cadence. In this tutorial, we will learn how to use Lightkurve to download these raw data, plot them, and understand their properties and units.

It is recommended that you first read the tutorial on how to use *Kepler* light curve products with Lightkurve. That tutorial will introduce you to some specifics of how *Kepler*, *K2*, and *TESS* make observations, and how these are displayed as light curves. It also introduces some important terms and concepts that are referred to in this tutorial.

*Kepler* observed a single field in the sky, although not all stars in this field were recorded. Instead, pixels were selected around certain targeted stars. This series of cutouts were downloaded and stored as an array of images in target pixel files, or TPFs. By summing up the amount of light (the *flux*) captured by the pixels in which the star appears, you can make a measurement of the brightness of a star over time.

TPFs are an important resource when studying an astronomical object with *Kepler*, *K2*, or *TESS*. The files allow us to understand the original images that were collected, and identify potential sources of noise or instrument-induced trends which may be less obvious in derived light curves. In this tutorial, we will use the *Kepler* mission as the main example, but these tools equally work for *TESS* and *K2*.

## Imports

This tutorial requires **[Lightkurve](https://docs.lightkurve.org)**, which in turn uses `matplotlib` for plotting.

In [ ]:
import lightkurve as lk
%matplotlib inline 

## 1. What is a Target Pixel File?

The target pixel file (TPF) of a star contains an image for each observing cadence, either a 30-minute Long Cadence or one-minute Short Cadence exposure in the case of *Kepler*. The files also include metadata detailing how the observation was made, as well as post-processing information such as the estimated intensity of the astronomical background in each image. (Read the [*Kepler* Archive Manual](https://archive.stsci.edu/files/live/sites/mast/files/home/missions-and-data/kepler/_documents/archive_manual.pdf), Section 2.3.2 for more information.)

TPFs are stored in a [FITS file format](https://fits.gsfc.nasa.gov/fits_primer.html). The Lightkurve package allows us to work with these binary files without having to worry about the details of the file structure. For examples on how to work with FITS files directly, read this tutorial on [Plotting Images from *Kepler* Target Pixel Files](https://github.com/spacetelescope/notebooks/blob/master/notebooks/MAST/Kepler/Kepler_TPF/kepler_tpf.ipynb).

## 2. Downloading a Target Pixel File

The TPFs of stars observed by the *Kepler* mission are stored on the [Mikulksi Archive for Space Telescopes](https://archive.stsci.edu/kepler/) (MAST) archive, along with metadata about the observations, such as which charge-coupled device (CCD) was used at each time.

Lightkurve's built-in tools allow us to search and download TPFs from the archive. As we did in the accompanying tutorial on light curves, we will start by downloading one quarter (a *Kepler* observing period approximately 90 days in duration) of *Kepler* data for the star named [Kepler-8](http://www.openexoplanetcatalogue.com/planet/Kepler-8%20b/), a star somewhat larger than the Sun, and the host of a [hot Jupiter planet](https://en.wikipedia.org/wiki/Hot_Jupiter).

Using the [search_targetpixelfile](https://docs.lightkurve.org/reference/api/lightkurve.search_targetpixelfile.html?highlight=search_targetpixelfile) function, we can find an itemized list of different TPFs available for Kepler-8.

In [ ]:
search_result = lk.search_targetpixelfile("Kepler-8", author="Kepler", cadence="long")
search_result

In this list, each row represents a different observing period. We find that *Kepler* recorded 18 quarters of data for this target across four years. The `search_targetpixelfile()` function takes several additional arguments, such as the `quarter` number or the `mission` name. You can find examples of its use in the [online documentation](https://docs.lightkurve.org/reference/api/lightkurve.search_targetpixelfile.html?highlight=search_targetpixelfile) for this function.

The search function returns a `SearchResult` object which has several convenient operations. For example, we can select the fourth data product in the list as follows:

In [ ]:
search_result[4]

We can download this data product using the [download()](https://docs.lightkurve.org/reference/api/lightkurve.SearchResult.download.html?highlight=download#lightkurve.SearchResult.download) method.

In [ ]:
tpf = search_result[4].download()

This instruction is identical to the following line:

In [ ]:
tpf = lk.search_targetpixelfile("Kepler-8", author="Kepler", cadence="long", quarter=4).download()

The `tpf_file` variable we have obtained in this way is a [KeplerTargetPixelFile](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.html?highlight=keplertargetpixelfile) object.

In [ ]:
tpf

This file object provides a convenient way to interact with the data file that has been returned by the archive, which contains both the TPF as well as metadata about the observations.

Before diving into the properties of the `KeplerTargetPixelFile`, we can plot the data, also using Lightkurve.

In [ ]:
%matplotlib inline
tpf.plot();

What you are seeing in this figure are pixels on the CCD camera, with which Kepler-8 was observed. The color indicates the amount of flux in each pixel, in electrons per second. The y-axis shows the pixel row, and the x-axis shows the pixel column. The title tells us the *Kepler* Input Catalogue (KIC) identification number of the target, and the observing cadence of this image. By default, `plot()` shows the first observation cadence in the quarter, but this can be changed by passing optional keyword arguments. You can type `help(tpf.plot)` to see a full list of those options.

### Note

You can also download TPF FITS files from the archive by hand, store them on your local disk, and open them using the `lk.read(<filename>)` function. This function will return a [KeplerTargetPixelFile](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.html?highlight=keplertargetpixelfile) object just as in the above example. You can find out where Lightkurve stored a given TPF by typing `tpf.path`:

In [ ]:
tpf.path

## 3. Accessing the Metadata

Our [KeplerTargetPixelFile](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.html?highlight=keplertargetpixelfile) includes the observation's metadata, loaded from the header of the TPF files downloaded from MAST. Many of these are similar to the metadata stored in the `KeplerLightCurve`, which are discussed in the accompanying tutorial.

The headers containing the metadata can be accessed from the [KeplerTargetPixelFile](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.html?highlight=keplertargetpixelfile) through the [get_header()](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.get_header.html?highlight=get_header) method.

For example, the first extension ("extension 0") of the file provides metadata related to the star, such as its magnitude in different passbands, its movement and position on the sky, and its location on *Kepler*'s CCD detector:

In [ ]:
tpf.get_header(ext=0)

This is an Astropy [`astropy.io.fits.Header`](https://docs.astropy.org/en/stable/io/fits/api/headers.html) object, which has many convenient features. For example, you can retrieve the value of an individual keyword as follows:

In [ ]:
tpf.get_header(ext=0).get('QUARTER')

When constructing a [KeplerTargetPixelFile](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.html?highlight=keplertargetpixelfile) from a FITS file, Lightkurve carries a subset of the metadata through into user-friendly object properties for convenience, which are available through shorthands (for example, `tpf.quarter`). You can view these properties with the the `show_properties()` method:

In [ ]:
tpf.show_properties()

A new piece of metadata not included in the `KeplerLightCurve` objects is the [World Coordinate System](https://fits.gsfc.nasa.gov/fits_wcs.html) (WCS). The WCS contains information about how pixel positions map to celestial sky coordinates. This is important when comparing a TPF from a *Kepler*, *K2*, or *TESS* observation to an observation of the same star with a different telescope.

You can access the WCS using [tpf.wcs](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.wcs.html?highlight=wcs), which is an Astropy WCS object:

In [ ]:
type(tpf.wcs)

For example, you can obtain the sky coordinates for the bottom left corner of the TPF as follows:

In [ ]:
tpf.wcs.pixel_to_world(0, 0)

Altogether, the metadata contains a lot of information, and you will rarely use it all, but it is important to know that it is available if you need it. For more details and a better overview of all of the metadata stored in a TPF, read the [*Kepler* Archive Manual](http://archive.stsci.edu/files/live/sites/mast/files/home/missions-and-data/kepler/_documents/archive_manual.pdf), specifically:
  - Section 2.3.2 Target Pixel Data
  - Appendix A.1: Target Pixel File Headers

## 4. Time, Flux, and Background

Finally, we have the most important properties of the TPF: the time and flux information. Just like a `KeplerLightCurve` object, we can access the time information as an Astropy `Time` object as follows:

In [ ]:
tpf.time

The pixel brightness data is available as an Astropy `Quantity` object named `tpf.flux`:

In [ ]:
tpf.flux

This object is a three-dimensional array, where each entry in the array represents one observing cadence. In our example, the flux array is composed of 4116 images, which are 5x5 pixels in size each:

In [ ]:
tpf.flux.shape

We can access the values of the first 5x5 pixel image as a NumPy array as follows:

In [ ]:
tpf.flux[0].value

At each cadence the TPF has four different flux-related data properties:

- `tpf.flux`: the stellar brightness after the background is removed.
- `tpf.flux_err`: the statistical uncertainty on the stellar flux after background removal.
- `tpf.flux_bkg`: the astronomical background brightness of the image.
- `tpf.flux_bkg_err`: the statistical uncertainty on the background flux.

All four of these data arrays are in units of electrons per second.

**Note**: for *Kepler*, the flux background isn't a measurement made using the local TPF data. Instead, at each cadence, the *Kepler* pipeline fits a model to thousands of empty pixels across each CCD in order to estimate a continuum background across the the CCD. For more details read the [*Kepler* Instrument Handbook](https://archive.stsci.edu/files/live/sites/mast/files/home/missions-and-data/kepler/_documents/KSCI-19033-002-instrument-hb.pdf), Section 2.6.2.4. In the case of *TESS*, local background pixels contained within a TPF are used instead.

**Note**: The `tpf.flux` values seen above have been quality-masked. This means that cadences of observations that violated the `quality_bitmask` parameter are removed, and so `tpf.flux` represents the data that you probably want to use to do your science. The `quality_bitmask` can also be accessed as a property of a [`KeplerTargetPixelFile`](https://docs.lightkurve.org/reference/api/lightkurve.KeplerTargetPixelFile.html?highlight=keplertargetpixelfile). For specific details on the `quality` flags, read the [*Kepler* Archive Manual](https://archive.stsci.edu/files/live/sites/mast/files/home/missions-and-data/k2/_documents/MAST_Kepler_Archive_Manual_2020.pdf), Section 2.3.1.1.

If you want to access flux and background flux measurements that *have not* been quality masked, you can pass a custom `quality_bitmask` parameter to the `download()` or `read()` method as follows:

In [ ]:
search = lk.search_targetpixelfile("Kepler-8", author="Kepler", cadence="long", quarter=4)
tpf = search.download(quality_bitmask=0)

You can see that the flux array of this object now has more cadences (4397) than the original one above (4116):

In [ ]:
tpf.flux.shape

Alternatively, we can access the unmasked contents of the original TPF FITS file at any time using the `hdu` property:

In [ ]:
tpf.hdu[1].data['FLUX'].shape

## About this Notebook

**Authors:** Oliver Hall (oliver.hall@esa.int), Geert Barentsen

**Updated On**: 2020-09-15

## Citing Lightkurve and Astropy

If you use `lightkurve` or `astropy` for published research, please cite the authors. Click the buttons below to copy BibTeX entries to your clipboard. 



In [ ]:
lk.show_citation_instructions()

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>
